## Data Cleaning

#### 1. Edit Names
#### 2. Fix Income
#### 3. Seperate Genre/Niche
#### 4. Separate Skills

In [85]:
import pandas as pd
import os 
import re

In [91]:
#os.chdir("..")

In [92]:
fields = ["Name", "Gender", "Age", "Nationality", "Years_Active", "Genre/Niche", "Beauty_Level", "Skills", "Education", "Award_Wins",
          "Media_Mentions","Social_Media_Followers", "Social_Media_Likes", "Network_Size", "MBTI", "Income", "Short_Story"]
data = pd.read_csv("data3000-adv-ds/chatgpt-influencer/data/raw/chatgpt_data.csv")
data.head()

,Name,Gender,Age,Nationality,Years_Active,Genre_Niche,Beauty_Level,Skills,Education,Award_Wins,Media_Mentions,Social_Media_Followers,Social_Media_Likes,Network_Size,MBTI_Personality,Income_Per_Year
0,Ava Johnson,Female,32,American,7,Lifestyle,8,"Fashion, Photography, Public Speaking",Bachelor's Degree in Marketing,3.0,500.0,1000000.0,50000.0,5.0,ENFJ,"$200,000"
1,Jackson Lee,Male,28,British,5,Fitness,7,"Personal Training, Nutrition, Motivational Spe...",High School Diploma,2.0,300.0,800000.0,40000.0,4.0,ISTP,"$150,000"
2,Sophie Garcia,Female,35,Spanish,8,Beauty & Makeup,9,"Makeup Artistry, Skincare, Content Creation",Master's Degree in Cosmetology,5.0,600.0,1200000.0,70000.0,6.0,INFJ,"$300,000"
3,Ryan Patel,Male,30,Indian,6,Food & Cooking,6,"Culinary Skills, Recipe Development, Food Phot...",Culinary School,3.0,400.0,900000.0,45000.0,5.0,ESTP,"$180,000"
4,Emma Chen,Female,25,Chinese,4,Travel & Adventure,7,"Photography, Writing, Adventure Sports",Bachelor's Degree in Journalism,2.0,250.0,600000.0,35000.0,4.0,ENFP,"$120,000"


### 0. Repeats?

In [113]:
data.duplicated().value_counts()

False    612
True     238
dtype: int64

In [116]:
data = data.drop_duplicates(keep = 'first')
data.describe()

,Age,Years_Active,Beauty_Level,Award_Wins,Media_Mentions,Social_Media_Followers,Social_Media_Likes,Network_Size,Income_Per_Year
count,612.000000,612.000000,612.000000,608.000000,608.000000,6.080000e+02,607.000000,607.000000,607.000000
mean,30.428105,7.070261,7.792484,2.847039,456.101974,9.641942e+05,53080.724876,4.461285,221004.942339
std,2.468073,1.229732,0.722160,0.706261,80.421682,1.484373e+05,10828.016808,0.540200,34899.359909
min,25.000000,4.000000,6.000000,1.000000,200.000000,1.000000e+02,25000.000000,3.000000,120000.000000
25%,29.000000,6.000000,7.000000,2.000000,400.000000,8.500000e+05,45000.000000,4.000000,200000.000000
50%,30.000000,7.000000,8.000000,3.000000,450.000000,9.500000e+05,50000.000000,4.000000,220000.000000
75%,32.000000,8.000000,8.000000,3.000000,500.000000,1.000000e+06,60000.000000,5.000000,240000.000000
max,40.000000,11.000000,9.000000,5.000000,700.000000,1.500000e+06,80000.000000,6.000000,400000.000000


### 1. Edit Names

In [117]:
def first_name(x):
    if ' ' in x["Name"]:
        sp = x["Name"].rfind(" ")
        return x["Name"][0:sp]

def last_name(x):
    if ' ' in x["Name"]:
        sp = x["Name"].rfind(" ")
        return x["Name"][sp:]

data["First_Name"] = data.apply(first_name, axis = 1)
data["Last_Name"] = data.apply(last_name, axis = 1)

data.head()


,Name,Gender,Age,Nationality,Years_Active,Genre_Niche,Beauty_Level,Skills,Education,Award_Wins,...,Network_Size,MBTI_Personality,Income_Per_Year,First_Name,Last_Name,Genre_Niche_1,Genre_Niche_2,Skill_1,Skill_2,Skill_3
0,Ava Johnson,Female,32,American,7,Lifestyle,8,"Fashion, Photography, Public Speaking",Bachelor's Degree in Marketing,3.0,...,5.0,ENFJ,200000.0,Ava,Johnson,Lifestyle,None,Fashion,Photography,Public Speaking
1,Jackson Lee,Male,28,British,5,Fitness,7,"Personal Training, Nutrition, Motivational Spe...",High School Diploma,2.0,...,4.0,ISTP,150000.0,Jackson,Lee,Fitness,None,Personal Training,Nutrition,Motivational Speaking
2,Sophie Garcia,Female,35,Spanish,8,Beauty & Makeup,9,"Makeup Artistry, Skincare, Content Creation",Master's Degree in Cosmetology,5.0,...,6.0,INFJ,300000.0,Sophie,Garcia,Beauty & Makeup,None,Makeup Artistry,Skincare,Content Creation
3,Ryan Patel,Male,30,Indian,6,Food & Cooking,6,"Culinary Skills, Recipe Development, Food Phot...",Culinary School,3.0,...,5.0,ESTP,180000.0,Ryan,Patel,Food & Cooking,None,Culinary Skills,Recipe Development,Food Photography
4,Emma Chen,Female,25,Chinese,4,Travel & Adventure,7,"Photography, Writing, Adventure Sports",Bachelor's Degree in Journalism,2.0,...,4.0,ENFP,120000.0,Emma,Chen,Travel & Adventure,None,Photography,Writing,Adventure Sports


### 2. Fix Income

In [118]:
def remove_extra(x):
    val = str(x["Income_Per_Year"])
    if "," in val:
        loc = val.rfind(",")
        first = val[1:loc]
        last = val[loc+1:]
        return int(first+last)

data["Income_Per_Year"] = data.apply(remove_extra, axis = 1)
data.head()

,Name,Gender,Age,Nationality,Years_Active,Genre_Niche,Beauty_Level,Skills,Education,Award_Wins,...,Network_Size,MBTI_Personality,Income_Per_Year,First_Name,Last_Name,Genre_Niche_1,Genre_Niche_2,Skill_1,Skill_2,Skill_3
0,Ava Johnson,Female,32,American,7,Lifestyle,8,"Fashion, Photography, Public Speaking",Bachelor's Degree in Marketing,3.0,...,5.0,ENFJ,None,Ava,Johnson,Lifestyle,None,Fashion,Photography,Public Speaking
1,Jackson Lee,Male,28,British,5,Fitness,7,"Personal Training, Nutrition, Motivational Spe...",High School Diploma,2.0,...,4.0,ISTP,None,Jackson,Lee,Fitness,None,Personal Training,Nutrition,Motivational Speaking
2,Sophie Garcia,Female,35,Spanish,8,Beauty & Makeup,9,"Makeup Artistry, Skincare, Content Creation",Master's Degree in Cosmetology,5.0,...,6.0,INFJ,None,Sophie,Garcia,Beauty & Makeup,None,Makeup Artistry,Skincare,Content Creation
3,Ryan Patel,Male,30,Indian,6,Food & Cooking,6,"Culinary Skills, Recipe Development, Food Phot...",Culinary School,3.0,...,5.0,ESTP,None,Ryan,Patel,Food & Cooking,None,Culinary Skills,Recipe Development,Food Photography
4,Emma Chen,Female,25,Chinese,4,Travel & Adventure,7,"Photography, Writing, Adventure Sports",Bachelor's Degree in Journalism,2.0,...,4.0,ENFP,None,Emma,Chen,Travel & Adventure,None,Photography,Writing,Adventure Sports


### 3. Separate Genre/Niche

In [119]:
def separate_genre(x, w):
    val = x["Genre_Niche"]
    if "," in val:
        loc = val.rfind("&")
        if w == "f":
            return val[0:loc]
        elif w == "l":
            return val[loc+1:]
    else:
        if w == "l":
            return None
        else:
            return val
    
data["Genre_Niche_1"] = data.apply(separate_genre, args = ("f"), axis = 1)
data["Genre_Niche_2"] = data.apply(separate_genre, args = ("l"), axis = 1)
data.head()
    

,Name,Gender,Age,Nationality,Years_Active,Genre_Niche,Beauty_Level,Skills,Education,Award_Wins,...,Network_Size,MBTI_Personality,Income_Per_Year,First_Name,Last_Name,Genre_Niche_1,Genre_Niche_2,Skill_1,Skill_2,Skill_3
0,Ava Johnson,Female,32,American,7,Lifestyle,8,"Fashion, Photography, Public Speaking",Bachelor's Degree in Marketing,3.0,...,5.0,ENFJ,None,Ava,Johnson,Lifestyle,None,Fashion,Photography,Public Speaking
1,Jackson Lee,Male,28,British,5,Fitness,7,"Personal Training, Nutrition, Motivational Spe...",High School Diploma,2.0,...,4.0,ISTP,None,Jackson,Lee,Fitness,None,Personal Training,Nutrition,Motivational Speaking
2,Sophie Garcia,Female,35,Spanish,8,Beauty & Makeup,9,"Makeup Artistry, Skincare, Content Creation",Master's Degree in Cosmetology,5.0,...,6.0,INFJ,None,Sophie,Garcia,Beauty & Makeup,None,Makeup Artistry,Skincare,Content Creation
3,Ryan Patel,Male,30,Indian,6,Food & Cooking,6,"Culinary Skills, Recipe Development, Food Phot...",Culinary School,3.0,...,5.0,ESTP,None,Ryan,Patel,Food & Cooking,None,Culinary Skills,Recipe Development,Food Photography
4,Emma Chen,Female,25,Chinese,4,Travel & Adventure,7,"Photography, Writing, Adventure Sports",Bachelor's Degree in Journalism,2.0,...,4.0,ENFP,None,Emma,Chen,Travel & Adventure,None,Photography,Writing,Adventure Sports


### 4. Separate Skills

In [120]:
def separate_skills(x, w):
    val = x["Skills"]
    s_comma = val.rfind(",")
    f_comma = val[0:s_comma-1].rfind(",")
    if w == "f":
        return val[0:f_comma]
    elif w == "s":
        return val[f_comma+1:s_comma]
    elif w == "t":
        return val[s_comma+1:]
    
data["Skill_1"] = data.apply(separate_skills, args = ("f"), axis = 1)
data["Skill_2"] = data.apply(separate_skills, args = ("s"), axis = 1)
data["Skill_3"] = data.apply(separate_skills, args = ("t"), axis = 1)
data.head()

,Name,Gender,Age,Nationality,Years_Active,Genre_Niche,Beauty_Level,Skills,Education,Award_Wins,...,Network_Size,MBTI_Personality,Income_Per_Year,First_Name,Last_Name,Genre_Niche_1,Genre_Niche_2,Skill_1,Skill_2,Skill_3
0,Ava Johnson,Female,32,American,7,Lifestyle,8,"Fashion, Photography, Public Speaking",Bachelor's Degree in Marketing,3.0,...,5.0,ENFJ,None,Ava,Johnson,Lifestyle,None,Fashion,Photography,Public Speaking
1,Jackson Lee,Male,28,British,5,Fitness,7,"Personal Training, Nutrition, Motivational Spe...",High School Diploma,2.0,...,4.0,ISTP,None,Jackson,Lee,Fitness,None,Personal Training,Nutrition,Motivational Speaking
2,Sophie Garcia,Female,35,Spanish,8,Beauty & Makeup,9,"Makeup Artistry, Skincare, Content Creation",Master's Degree in Cosmetology,5.0,...,6.0,INFJ,None,Sophie,Garcia,Beauty & Makeup,None,Makeup Artistry,Skincare,Content Creation
3,Ryan Patel,Male,30,Indian,6,Food & Cooking,6,"Culinary Skills, Recipe Development, Food Phot...",Culinary School,3.0,...,5.0,ESTP,None,Ryan,Patel,Food & Cooking,None,Culinary Skills,Recipe Development,Food Photography
4,Emma Chen,Female,25,Chinese,4,Travel & Adventure,7,"Photography, Writing, Adventure Sports",Bachelor's Degree in Journalism,2.0,...,4.0,ENFP,None,Emma,Chen,Travel & Adventure,None,Photography,Writing,Adventure Sports


In [121]:
#os.chdir("..")
os.getcwd()

'/Users/sophiacolonello/Desktop/school/junior year'

In [122]:
data.to_csv("second semester/data3000-adv-ds/chatgpt-influencer/data/processed/chatgpt_data.csv")